In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS WITH 
product_mdl AS (
    SELECT a.*,
        CASE 
            WHEN manufacturer_id IN ('1915', '2325') THEN 'MML'
            WHEN product_id IN ( "10184334","10184357","10184356","10184358") OR (manufacturer_id ='1012' AND brand_name IN ('HEO CAO BỒI','PONNIE')) THEN 'MCH'
            ELSE 'OTHER' 
        END product_source
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product a
    WHERE
        product_id IN (SELECT DISTINCT product_id FROM {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_winare_mdl_special_sku_add)
        OR (manufacturer_id IN ('1915', '2325') AND NOT(sub_manufacturer_id = '19150005' AND mch5_id = '102050404'))
        OR (manufacturer_id = '1012' AND brand_name IN ('HEO CAO BỒI','PONNIE'))
),
store_sku AS (
    SELECT
        a.store_id,
        a.product_id,
        c.business_unit,
        c.city,
        c.store_name,
        c.store_open_date,
        IFNULL(IF(c.store_open_date > CURRENT_DATE - 14, 'YES', 'NO'),'NO') is_new_store,
        c.size_v,
        b.manufacturer_name,
        b.product_name,
        b.base_uom
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active a
        LEFT JOIN product_mdl b ON a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
        LEFT JOIN 
            (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_mdl_winlife_asm_dr_tracking WHERE is_winare = 'YES') d 
            ON a.store_id = d.store_id AND a.product_id = d.product_id
    WHERE 
        (b.product_source  = 'MCH'
        OR (b.product_source = 'MML' AND mch3_id <> '10104')
        OR (b.product_source = 'MML' AND d.product_id IS NOT NULL)
        OR  b.product_source = 'OTHER')
        AND c.business_unit IN ('1500', '2000')
        AND c.store_status_desc IN ("Đang hoat động","Đang hoạt động","Sắp đóng cửa")
        AND UPPER(c.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(c.store_name) NOT LIKE 'OUTLET%'  
        AND UPPER(c.store_name) NOT LIKE '%HUB%'
        AND UPPER(c.store_name) NOT LIKE '%ONLINE%'
),
table_date AS (
    SELECT explode(sequence(date_sub(trunc(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 'WEEK'), 14),date_sub(trunc(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 'WEEK'), 1) )) AS calday
),
base_sale_qty AS (
    SELECT
        DATE_TRUNC(a2.calday, 'WEEK') calweek,
        a1.store_id,
        a1.product_id,
        SUM(IFNULL(base_sale_qty, 0)) base_sale_qty
    FROM
        store_sku a1 CROSS JOIN table_date a2
        LEFT JOIN (
            SELECT calday, store_id, product_id, revenue.base_sale_qty base_sale_qty FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
            WHERE calday BETWEEN DATE_SUB(TRUNC(DATE(CURRENT_DATE() + INTERVAL 7 HOURS), 'WEEK'), 14) AND DATE_SUB(TRUNC(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 'WEEK'), 1)
        ) b ON a2.calday = b.calday AND a1.store_id = b.store_id AND  a1.product_id = b.product_id
    GROUP BY 1,2,3
),
base_sale_qty_weekly AS (
    SELECT
        *,
        'W' || ROW_NUMBER() OVER (PARTITION BY store_id, product_id ORDER BY calweek DESC) week_name
    FROM base_sale_qty
),
stock AS (
    SELECT
        b.store_id,
        b.product_id,
        SUM(a.closing_stock_qty) closing_stock_qty
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_hourly a
        INNER JOIN store_sku b ON CAST(a.store_id AS STRING) = b.store_id AND CAST(a.product_id AS STRING) = b.product_id
    WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND calhour = 6
    GROUP BY 1,2
),
stock_24h AS (
    SELECT
        b.store_id,
        b.product_id,
        SUM(a.closing_stock_quantity) closing_stock_quantity
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_1000 a
        INNER JOIN store_sku b ON CAST(a.store_id AS STRING) = b.store_id AND CAST(a.product_id AS STRING) = b.product_id
    WHERE calday = DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1) 
    GROUP BY 1,2
),
open_order AS (
    SELECT
        a.store_id,
        a.product_id,
        IFNULL(SUM(a.po_qty * c.numerator/c.denomintR), 0) open_order
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
        INNER JOIN store_sku b ON a.store_id = b.store_id AND a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
    WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 3) AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 0)
        AND (a.compl_del <> 'X' OR a.compl_del IS NULL)
        AND (a.po_gr_date IS NULL)
        AND (a.gr_date IS NULL)
        AND (a.webportal_status NOT IN ('Đã hủy', 'Đã giao hàng', 'Quá hạn') OR a.webportal_status IS NULL)
    GROUP BY 1,2
),
order_today AS (
    SELECT
        a.store_id,
        a.product_id,
        IFNULL(SUM(a.po_qty * c.numerator/c.denomintR), 0) open_order
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
        INNER JOIN store_sku b ON a.store_id = b.store_id AND a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.product_id = c.material AND a.po_unit = c.mat_unit
    WHERE a.creation_date BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 0) AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 0)
    GROUP BY 1,2
),
product_unit AS (
    SELECT 
        a.*,  
        b.base_uom,
        CASE 
         WHEN a.delivery_unit = b.base_uom THEN 1 
         WHEN a.delivery_unit <> 'KG' AND b.base_uom = 'KG' THEN TRY_DIVIDE(1,a.convert_pack_kg)
         WHEN a.delivery_unit = 'KG' AND b.base_uom <> 'KG' THEN a.convert_pack_kg
        END conversion_rate
    FROM  {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_coldchain_upload_mdl_pack_size  a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b USING (product_id)
),
stock_clearance AS (
    SELECT
       a.calday,
       a.store_id,
       a.product_id,
       CASE 
        WHEN b.base_uom = 'KG' THEN SUM(pack_qty) *c.convert_pack_kg
        ELSE SUM(pack_qty) 
       END  stock_clearance
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_campaign_inventory a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b USING (product_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_coldchain_upload_mdl_pack_size c ON a.product_id = c.product_id
    WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    GROUP BY 1,2,3,base_uom, convert_pack_kg
),
sale_cover_leadtime AS (
    SELECT
        store_id,
        product_id,
        SUM(sale_cover_leadtime_fc) sale_cover_leadtime_fc
    FROM
        (
            SELECT
                a.store_id,
                a.product_id,
                SUM(fc_sold_qty) sale_cover_leadtime_fc
            FROM
                {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale_by_dayofweek a
                LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
                LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule c ON b.city = c.city
            WHERE a.dayofweek BETWEEN EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) AND EXTRACT(DAYOFWEEK FROM (c.first_delivery_day -1))
            AND EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) <= EXTRACT(DAYOFWEEK FROM (c.first_delivery_day -1))
            GROUP BY 1,2
            UNION ALL
            SELECT
                a.store_id,
                a.product_id,
                SUM(fc_sold_qty) sale_cover_leadtime_fc
            FROM
                {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale_by_dayofweek a
                LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
                LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule c ON b.city = c.city
            WHERE NOT(a.dayofweek > EXTRACT(DAYOFWEEK FROM (c.first_delivery_day -1)) AND a.dayofweek < EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)))
            AND EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) > EXTRACT(DAYOFWEEK FROM (c.first_delivery_day -1))
            GROUP BY 1,2
        )
    GROUP BY 1,2
),
sale_cover_delivery AS (
    SELECT
        store_id,
        product_id,
        SUM(sale_cover_delivery_fc) sale_cover_delivery_fc
    FROM (
        SELECT
            a.store_id,
            a.product_id,
            SUM(fc_sold_qty) sale_cover_delivery_fc
        FROM
            {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale_by_dayofweek a
                LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
            LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule c ON b.city = c.city
        WHERE a.DAYOFWEEK BETWEEN dayofweek(c.FIRST_DELIVERY_DAY) AND dayofweek(date_sub(c.SECOND_DELIVERY_DAY, 1))     AND dayofweek(c.FIRST_DELIVERY_DAY) <= dayofweek(date_sub(c.SECOND_DELIVERY_DAY, 1))
        GROUP BY 1,2
        UNION ALL
        SELECT
            a.store_id,
            a.product_id,
            SUM(fc_sold_qty) sale_cover_delivery_fc
        FROM
            {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale_by_dayofweek a
                LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
            LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule c ON b.city = c.city
        WHERE NOT(a.DAYOFWEEK > dayofweek(date_sub(c.SECOND_DELIVERY_DAY, 1)) AND a.DAYOFWEEK < dayofweek(c.FIRST_DELIVERY_DAY))
        AND dayofweek(c.FIRST_DELIVERY_DAY) > dayofweek(date_sub(c.SECOND_DELIVERY_DAY, 1))
        GROUP BY 1,2
    )
    GROUP BY 1,2
),
temp_0 AS (
    SELECT
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)-1 calday,
        a.store_id,
        a.product_id,
        a.store_open_date,
        a.is_new_store,
        a.size_v,
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) current_date,
        CASE
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 1) THEN 'SUN'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 2) THEN 'MON'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 3) THEN 'TUE'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 4) THEN 'WED'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 5) THEN 'THU'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 6) THEN 'FRI'
            WHEN (SELECT EXTRACT(DAYOFWEEK FROM DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)) = 7) THEN 'SAT'
        END current_weekday,
        IF(business_unit = '1500', 'WMT', IF(business_unit = '2000', 'WMP', 'OTHER')) business_unit,
        a.city,
        a.store_name,
        a.manufacturer_name,
        a.product_name,
        a.base_uom base_unit,
        IFNULL(b.closing_stock_qty, 0) closing_stock_qty,
        IFNULL(b1.closing_stock_quantity,0) closing_stock_qty_24h_ld,
        IFNULL(c.open_order, 0) open_order,
        IFNULL(c2.open_order,0) order_today,
        IFNULL(c1.stock_clearance,0) stock_clearance,
        IFNULL(d.avg_sold_qty_28d , 0) avg_sold_qty_28d,
        IFNULL(d.avg_sold_qty_4mon, 0) avg_sold_qty_4mon,
        IFNULL(d.avg_sold_qty_4tue, 0) avg_sold_qty_4tue,
        IFNULL(d.avg_sold_qty_4wed, 0) avg_sold_qty_4wed,
        IFNULL(d.avg_sold_qty_4thu, 0) avg_sold_qty_4thu,
        IFNULL(d.avg_sold_qty_4fri, 0) avg_sold_qty_4fri,
        IFNULL(d.avg_sold_qty_4sat, 0) avg_sold_qty_4sat,
        IFNULL(d.avg_sold_qty_4sun, 0) avg_sold_qty_4sun,
        IFNULL(d.fc_sold_qty_28d,0) fc_sold_qty_28d,
        IFNULL(d.fc_sold_qty_mon,0) fc_sold_qty_mon,
        IFNULL(d.fc_sold_qty_tue,0) fc_sold_qty_tue,
        IFNULL(d.fc_sold_qty_wed,0) fc_sold_qty_wed,
        IFNULL(d.fc_sold_qty_thu,0) fc_sold_qty_thu,
        IFNULL(d.fc_sold_qty_fri,0) fc_sold_qty_fri,
        IFNULL(d.fc_sold_qty_sat,0) fc_sold_qty_sat,
        IFNULL(d.fc_sold_qty_sun,0) fc_sold_qty_sun,
        IFNULL(e.delivery_unit, a.base_uom) delivery_unit,
        IFNULL(e.conversion_rate,1) conversion_rate,
        f.is_order_date,
        f.delivery_schedule,
        f.send_fc_date,
        f.order_schedule,
        f.first_delivery_day,
        f.second_delivery_day,
        IFNULL(g.sale_cover_leadtime_fc, 0) sale_cover_leadtime_fc,
        IFNULL(h.sale_cover_delivery_fc, 0) sale_cover_delivery_fc,
        i1.base_sale_qty sale_qty_w1,
        i2.base_sale_qty sale_qty_w2
    FROM store_sku a
        LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.d_fnr_fresh_delivery_schedule f ON a.city = f.city
        LEFT JOIN {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale d ON a.store_id = d.store_id AND a.product_id = d.product_id
        LEFT JOIN stock b ON a.store_id = b.store_id AND a.product_id = b.product_id
        LEFT JOIN stock_24h b1 ON a.store_id = b1.store_id AND a.product_id = b1.product_id
        LEFT JOIN open_order c ON a.store_id = c.store_id AND a.product_id = c.product_id
        LEFT JOIN order_today c2 ON a.store_id = c2.store_id AND a.product_id = c2.product_id
        LEFT JOIN stock_clearance c1 ON a.store_id = c1.store_id AND a.product_id = c1.product_id
        LEFT JOIN product_unit e ON a.product_id = e.product_id
        LEFT JOIN sale_cover_leadtime g ON a.store_id = g.store_id AND a.product_id = g.product_id
        LEFT JOIN sale_cover_delivery h ON a.store_id = h.store_id AND a.product_id = h.product_id
        LEFT JOIN base_sale_qty_weekly i1 ON a.store_id = i1.store_id AND a.product_id = i1.product_id AND i1.week_name = 'W1'
        LEFT JOIN base_sale_qty_weekly i2 ON a.store_id = i2.store_id AND a.product_id = i2.product_id AND i2.week_name = 'W2'
),
temp_1 AS (
    SELECT a.*,
        (
            IF(a.closing_stock_qty < 0 OR a.closing_stock_qty IS NULL, 0, a.closing_stock_qty)
            + IF(a.open_order < 0 OR a.open_order IS NULL, 0, a.open_order)
            - stock_clearance
            - IF(a.sale_cover_leadtime_fc < 0 OR a.sale_cover_leadtime_fc IS NULL, 0, a.sale_cover_leadtime_fc)
        ) open_stock_at_first_delivery
    FROM temp_0 a
),
temp_2 AS (
    SELECT a.* EXCEPT(open_stock_at_first_delivery),
        + GREATEST(IFNULL(e.mdq,0),fc_sold_qty_28d * IFNULL(d.safety_stock_day,1)) safety_stock_demand,
        IF(a.open_stock_at_first_delivery < 0, 0, a.open_stock_at_first_delivery) open_stock_at_first_delivery,
        (
            sale_cover_delivery_fc
            + GREATEST(IFNULL(e.mdq,0),fc_sold_qty_28d * IFNULL(d.safety_stock_day,1))
            - IF(a.open_stock_at_first_delivery < 0, 0, a.open_stock_at_first_delivery)
        ) po_qty_base_unit
    FROM temp_1 a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING (store_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c USING (product_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_winare_mdl_safetystock_bycity d 
              ON b.city = d.city  AND IF(c.mch2_id='101',c.mch2_id,'102') = d.mch2_id
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_mdl_winlife_asm_dr_tracking e ON a.store_id =  e.store_id AND a.product_id = e.product_id              
),
temp_3 AS (
    SELECT a.* EXCEPT(po_qty_base_unit),
        TRY_DIVIDE(open_stock_at_first_delivery, fc_sold_qty_28d) AS open_dio_at_first_delivery,
        IF(a.po_qty_base_unit < 0, 0, a.po_qty_base_unit) po_qty_base_unit,
        IF(delivery_unit <> 'KG',CEIL(IF(a.po_qty_base_unit < 0, 0, a.po_qty_base_unit) * conversion_rate),
                                IF(a.po_qty_base_unit < 0, 0, a.po_qty_base_unit) * conversion_rate) po_qty_delivery_unit
    FROM temp_2 a
),
main AS (
    SELECT a.* EXCEPT(po_qty_delivery_unit), 
        CASE WHEN (e.pack_size IS NULL OR po_qty_delivery_unit = 0 OR a.delivery_unit = 'KG') THEN po_qty_delivery_unit
             WHEN po_qty_delivery_unit <= e.pack_size THEN e.pack_size
             WHEN po_qty_delivery_unit > e.pack_size THEN ROUND(TRY_DIVIDE(po_qty_delivery_unit,e.pack_size),0)*pack_size END po_qty_delivery_unit
    FROM temp_3 a 
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_coldchain_upload_mdl_pack_size e ON a.product_id = e.product_id
),
po_group AS (
    SELECT
        DISTINCT
        store_id,
        calday,
        manufacturer_name
    FROM main
    WHERE 1=1
    AND  is_order_date = "YES"
    ORDER BY 1,3
),
po_group2 AS (
    SELECT
        store_id, calday, manufacturer_name,
        ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS rn
    FROM po_group
),
max_po_rn AS (
    SELECT IFNULL(MAX(rn),0)  AS max_rn
    FROM {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli_his
),
po_group3 AS (
    SELECT store_id, calday, manufacturer_name,rn + max_rn AS rn
    FROM po_group2, max_po_rn
)
SELECT m.*,
    TRY_DIVIDE(open_stock_at_first_delivery + po_qty_delivery_unit*conversion_rate, fc_sold_qty_28d) AS dio_after_receipt,
    CONCAT('AM', LPAD(CAST(rn AS STRING),9,'0')) AS gcp_id,rn,
    a.region_dc,
    b.mdq,
    CASE
        WHEN m.delivery_unit = 'KG' THEN ROUND(TRY_DIVIDE(m.po_qty_delivery_unit, c.convert_pack_kg), 0)
        WHEN m.delivery_unit IN ('G1', 'KH') THEN m.po_qty_delivery_unit
        ELSE NULL
    END po_qty_pack
FROM main m
LEFT JOIN (SELECT DISTINCT city, region_dc FROM {catalog_name}.udp_wcm_gold_db_winare_fresh.d_mapping_dc_store) a ON a.city = m.city
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_mdl_winlife_asm_dr_tracking b ON m.store_id = b.store_id AND m.product_id = b.product_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_coldchain_upload_mdl_pack_size c ON m.product_id = c.product_id 
LEFT JOIN po_group3 p
ON m.store_id = p.store_id
AND m.calday = p.calday
AND m.manufacturer_name = p.manufacturer_name
AND m.is_order_date = "YES";
""")

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli_his (
    calday DATE,
    store_id STRING,
    product_id STRING,
    store_open_date DATE,
    is_new_store STRING,
    size_v STRING,
    current_date DATE,
    current_weekday STRING,
    business_unit STRING,
    city STRING,
    store_name STRING,
    manufacturer_name STRING,
    product_name STRING,
    base_unit STRING,
    closing_stock_qty DOUBLE,
    closing_stock_qty_24h_ld DOUBLE,
    open_order DOUBLE,
    order_today DOUBLE,
    stock_clearance DOUBLE,
    avg_sold_qty_28d DOUBLE,
    avg_sold_qty_4mon DOUBLE,
    avg_sold_qty_4tue DOUBLE,
    avg_sold_qty_4wed DOUBLE,
    avg_sold_qty_4thu DOUBLE,
    avg_sold_qty_4fri DOUBLE,
    avg_sold_qty_4sat DOUBLE,
    avg_sold_qty_4sun DOUBLE,
    fc_sold_qty_28d DOUBLE,
    fc_sold_qty_mon DOUBLE,
    fc_sold_qty_tue DOUBLE,
    fc_sold_qty_wed DOUBLE,
    fc_sold_qty_thu DOUBLE,
    fc_sold_qty_fri DOUBLE,
    fc_sold_qty_sat DOUBLE,
    fc_sold_qty_sun DOUBLE,
    delivery_unit STRING,
    conversion_rate DOUBLE,
    is_order_date STRING,
    delivery_schedule STRING,
    send_fc_date LONG,
    order_schedule STRING,
    first_delivery_day DATE,
    second_delivery_day DATE,
    sale_cover_leadtime_fc DOUBLE,
    sale_cover_delivery_fc DOUBLE,
    sale_qty_w1 DOUBLE,
    sale_qty_w2 DOUBLE,
    safety_stock_demand DOUBLE,
    open_stock_at_first_delivery DOUBLE,
    open_dio_at_first_delivery DOUBLE,
    po_qty_base_unit DOUBLE,
    po_qty_delivery_unit DOUBLE,
    dio_after_receipt DOUBLE,
    gcp_id STRING,
    rn LONG,
    region_dc STRING,
    mdq DOUBLE,
    po_qty_pack DOUBLE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli_his
WHERE CALDAY = CURRENT_DATE-1;
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli_his
SELECT calday, store_id, product_id, store_open_date, is_new_store, size_v, current_date, current_weekday, business_unit, city, store_name, manufacturer_name, product_name, base_unit, closing_stock_qty, closing_stock_qty_24h_ld, open_order, order_today, stock_clearance, avg_sold_qty_28d, avg_sold_qty_4mon, avg_sold_qty_4tue, avg_sold_qty_4wed, avg_sold_qty_4thu, avg_sold_qty_4fri, avg_sold_qty_4sat, avg_sold_qty_4sun, fc_sold_qty_28d, fc_sold_qty_mon, fc_sold_qty_tue, fc_sold_qty_wed, fc_sold_qty_thu, fc_sold_qty_fri, fc_sold_qty_sat, fc_sold_qty_sun, delivery_unit, conversion_rate, is_order_date, delivery_schedule, send_fc_date, order_schedule, first_delivery_day, second_delivery_day, sale_cover_leadtime_fc, sale_cover_delivery_fc, sale_qty_w1, sale_qty_w2, safety_stock_demand, open_stock_at_first_delivery, open_dio_at_first_delivery, po_qty_base_unit, po_qty_delivery_unit, dio_after_receipt, gcp_id, rn, region_dc, mdq, po_qty_pack 
FROM {catalog_name}.udp_wcm_gold_db_supra_fresh.a_daily_are_meatdeli;       
""")